In [2]:
toyota_df.head()

Recalcular age con mes y anio de fabricacion

In [ ]:
toyota_df.describe().T

In [ ]:
toyota_df.shape

In [ ]:
toyota_df.info()

In [ ]:
toyota_df.isnull().sum()

In [ ]:
#Importing libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# Search for duplicates
toyota_df.duplicated().value_counts()

## Analisis de modelo

Listar modelos del dataset.

In [ ]:
toyota_df['Model'].value_counts()

Hay valores mal escritos, se debe realizar limpieza.

In [ ]:
toyota_df['Model'] = toyota_df['Model'].str.upper()
toyota_df['Model'] = toyota_df['Model'].str.strip()
toyota_df['Model'].value_counts()

Vamos a sacar signos de pregunta en todos. Patron comun que nombres inician con ?

In [ ]:
toyota_df['Model'] = toyota_df['Model'].str.replace('?', '')
toyota_df['Model'].value_counts()

Existen varios modelos iguales escritos de forma diferente. Sin embargo, en la mayoria de los casos, los valores se diferencian por cuestiones como la cantidad de puertas o detalles de algun agregado. Es necesario usar esta columna de modelos o podemos reemplazarla con otras columnas? Vamos a eliminar cadenas innecesarias en los nombres de los modelos.

In [ ]:
toyota_df['Model'] = toyota_df['Model'].str.replace('TOYOTA COROLLA', '')
toyota_df['Model'] = toyota_df['Model'].str.replace('LINEA', '')
toyota_df['Model'] = toyota_df['Model'].str.replace('HATCHBACK', 'HATCHB')
toyota_df['Model'] = toyota_df['Model'].str.replace(' HB', 'HATCHB')
toyota_df['Model'] = toyota_df['Model'].str.replace('LIFTBACK', 'LIFTB')
toyota_df['Model'] = toyota_df['Model'].str.replace('LB', 'LIFTB')
toyota_df['Model'] = toyota_df['Model'].str.replace('VVT I', 'VVTI')
toyota_df['Model'] = toyota_df['Model'].str.replace('VVT-I', 'VVTI')
toyota_df['Model'] = toyota_df['Model'].str.replace('VVTL-I', 'VVTLI')
toyota_df['Model'] = toyota_df['Model'].str.replace('VVTL I', 'VVTLI')
toyota_df['Model'] = toyota_df['Model'].str.replace('T-SPORT', 'TSPORT')
toyota_df['Model'] = toyota_df['Model'].str.replace('T SPORT', 'TSPORT')

# Delete doors because it's included as a feature
toyota_df['Model'] = toyota_df['Model'].str.replace(r'\d+\/\d+\-DOORS', '', regex=True)
# Delete cc because it's included as a feature
toyota_df['Model'] = toyota_df['Model'].str.replace(r'\d+\.\d+', '', regex=True)

#Let's make an array with all the words remaining in the column
words = toyota_df['Model'].str.split().explode()
words.unique()

### Gamas 

De las palabras vistas arriba, 'Terra', 'Luna' y 'Sol' son gamas de toyota corollas que ofrecian diferentes caracteristicas. Vamos a crear una columna en donde podamos visualizar los tipos de cada instancia.

In [ ]:
toyota_df['Model_type'] = toyota_df['Model'].apply(lambda x: 0 if 'TERRA' in x else 1 if 'LUNA' in x else 2 if 'SOL' in x else 3)
sns.histplot(toyota_df['Model_type'])
plt.show()


In [ ]:
sns.scatterplot(x='Model_type', y='Price', data=toyota_df)
plt.show()


Analicemos que pasa con los que no tienen una de las categorias.

In [ ]:
def get_not_classified(df, column, value):
    return df[df[column] == value]

In [ ]:
df = get_not_classified(toyota_df, 'Model_type', 3)
df.head()

Mucho TSPORT, vamos a ver si esto corresponde a otra categoria diferente que no habiamos tenido en cuenta.

In [ ]:
def classified_contains(df, column, value, contains):
    temp_df = df[df[column] != value]
    temp_df['Model'].str.contains(contains)
    temp_df.value_counts()

In [ ]:
classified_contains(toyota_df, 'Model_type', 3, 'TSPORT')

Nunca aparece el valor de TSPORT cuando hay presente uno de los modelos que tomamos, entendemos que se trata de otro modelo

In [ ]:
toyota_df.drop(['Model_type'], axis=1, inplace=True)
toyota_df['Model_type'] = toyota_df['Model'].apply(lambda x: 0 if 'TERRA' in x else 1 if 'LUNA' in x else 2 if 'SOL' in x else 3 if 'TSPORT' in x else 4)
sns.histplot(toyota_df['Model_type'])
plt.show()

Repetimos el analisis para ver que queda por clasificar

In [ ]:
df = get_not_classified(toyota_df, 'Model_type', 4)
df.head()

Parece ser que el resto no esta clasificado en el dataset. Debemos evaluar que hacemos con esos.

In [ ]:
sns.scatterplot(x='Model_type', y='Price', data=toyota_df)
plt.show()

Algo interesante es la palabra comfort, que solo aparece en modelos terra. Quizas es otra categoria.

In [ ]:
# Create a new vector with true or false if the model contains comfort and not terra
comfort_and_not_terra = toyota_df['Model'].str.contains('COMF') & ~toyota_df['Model'].str.contains('TERRA')
comfort_and_not_terra.value_counts()

In [ ]:
toyota_df.drop(['Model_type'], axis=1, inplace=True)
toyota_df['Model_type'] = toyota_df['Model'].apply(lambda x: 0 if 'TERRA' in x else 1 if 'COMF' in x else 2 if 'LUNA' in x else 3 if 'SOL' in x else 4 if 'TSPORT' in x else 5)
sns.histplot(toyota_df['Model_type'])
plt.show()

In [ ]:
sns.scatterplot(x='Model_type', y='Price', data=toyota_df)
plt.show()

### Carroceria

Hay por lo menos tres tipos de carroceria que detectamos en los nombres: liftback, hatchback y wagon. Vamos a clasificarlos

In [ ]:
toyota_df['carroceria'] = toyota_df['Model'].apply(lambda x: 0 if 'HATCHB' in x else 1 if 'LIFTB' in x else 2 if 'WAGON' in x else 3)
sns.histplot(toyota_df['carroceria'])
plt.show()

In [ ]:
df = get_not_classified(toyota_df, 'carroceria', 3)
df.head()

El resto parece no estar clasificado.

In [ ]:
sns.scatterplot(x='carroceria', y='Price', data=toyota_df)
plt.show()

No parece tener mucha relacion con el precio.

### Tecnologia motor

In [ ]:
toyota_df['tecnologia_motor'] = toyota_df['Model'].apply(lambda x: 0 if 'VVTI' in x else 1 if 'VVTLI' in x else 2)
sns.histplot(toyota_df['tecnologia_motor'])
plt.show()

In [ ]:
df = get_not_classified(toyota_df, 'tecnologia_motor', 2)
df.head()

In [ ]:
sns.scatterplot(x='tecnologia_motor', y='Price', data=toyota_df)
plt.show()

In [ ]:
toyota_df.drop(['Model'], axis=1, inplace=True)

## Analisis combustible

In [ ]:
toyota_df['Fuel_Type'].value_counts()

Parece correcto. No hay categorias anormales. Los convertimos a numeros y vemos el scatterplot

In [ ]:
# Map fuel type to numbers
toyota_df['Fuel_Type'] = toyota_df['Fuel_Type'].map({'Petrol': 0, 'Diesel': 1, 'CNG': 2})
sns.scatterplot(x='Fuel_Type', y='Price', data=toyota_df)
plt.show()

## Analsis generales

In [ ]:
toyota_df.drop(['Id'], axis=1, inplace=True)

In [ ]:
# Let's create histograms for each column with seaborn
plt.figure(figsize=(10, 10))
toyota_df.hist(figsize=(10, 10))
plt.show()

In [ ]:
toyota_df['Cylinders'].value_counts()

Solo hay un valor para esta feature, no sirve para predecir nada.

In [ ]:
toyota_df.drop(['Cylinders'], axis=1, inplace=True)

In [ ]:
correlation_matrix = toyota_df.corr(method='pearson')